In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [5]:
# Загрузка данных
(x_train, _), (x_test, _) = mnist.load_data()

# Нормализация и изменение формы данных
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Определение архитектуры автоэнкодера
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

# Создание модели автоэнкодера
autoencoder = Model(input_img, decoded)

# Компиляция модели
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Обучение автоэнкодера
autoencoder.fit(x_train, x_train,
                epochs=15,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/15
235/235 [==============================] - 6s 20ms/step - loss: 0.2435 - val_loss: 0.1631
Epoch 2/15
235/235 [==============================] - 4s 16ms/step - loss: 0.1480 - val_loss: 0.1339
Epoch 3/15
235/235 [==============================] - 3s 15ms/step - loss: 0.1291 - val_loss: 0.1219
Epoch 4/15
235/235 [==============================] - 4s 17ms/step - loss: 0.1199 - val_loss: 0.1152
Epoch 5/15
235/235 [==============================] - 4s 18ms/step - loss: 0.1142 - val_loss: 0.1100
Epoch 6/15
235/235 [==============================] - 4s 16ms/step - loss: 0.1099 - val_loss: 0.1073
Epoch 7/15
235/235 [==============================] - 4s 15ms/step - loss: 0.1067 - val_loss: 0.1041
Epoch 8/15
235/235 [==============================] - 5s 20ms/step - loss: 0.1039 - val_loss: 0.1013
Epoch 9/15
235/235 [==============================] - 4s 15ms/step - loss: 0.1017 - val_loss: 0.0991
Epoch 10/15
235/235 [==============================] - 3s 15ms/step - loss: 0.0994 - val_lo

In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

In [10]:
# Архитектура VAE
class VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(VariationalAutoencoder, self).__init__()
        # Энкодер
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.ReLU(),
        )
        # Среднее и логарифм дисперсии
        self.fc_mu = nn.Linear(400, 20)
        self.fc_logvar = nn.Linear(400, 20)
        # Декодер
        self.decoder = nn.Sequential(
            nn.Linear(20, 400),
            nn.ReLU(),
            nn.Linear(400, 28*28),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = self.fc_mu(h), self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Преобразования для изображений
transform = transforms.Compose([transforms.ToTensor()])

# Загрузка данных
train_data = datasets.MNIST('.', download=True, train=True, transform=transform)
test_data = datasets.MNIST('.', download=True, train=False, transform=transform)

# Создание загрузчиков данных
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

# Оптимизатор и функция потерь
vae = VariationalAutoencoder()
optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)

# Функция потерь для VAE
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Обучение модели
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset)}')

# Обучение модели
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(epoch)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 142420616.62it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 21364319.90it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 91397835.24it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5485323.57it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



====> Epoch: 1 Average loss: 147.2773332763672
====> Epoch: 2 Average loss: 116.03311767171223
====> Epoch: 3 Average loss: 111.47246021728516
====> Epoch: 4 Average loss: 109.41965707194011
====> Epoch: 5 Average loss: 108.23862318929037
====> Epoch: 6 Average loss: 107.37891144205729
====> Epoch: 7 Average loss: 106.75468869628907
====> Epoch: 8 Average loss: 106.29840393880208
====> Epoch: 9 Average loss: 105.91854376627604
====> Epoch: 10 Average loss: 105.52163716634115
